In [1]:
# Libraries

import numpy as np
import pandas as pd
import re
import emoji

import requests
from bs4 import BeautifulSoup
import html
import lxml

In [2]:
# Link to the event
url = "https://feverup.com/m/100122"
r = requests.get(url)
# Pulling the data from the link
soup = BeautifulSoup(r.text, "lxml")
# Take event title and description
# In the event description is where "accesibility" info is located
event_title = soup.find(class_ = "plan-hero__title").text
event_descr = soup.find(class_ = "plan-description mb-32")
# Transform the event description into string for later processing
event_descr = str(event_descr)
# Get price info
event_price = soup.find(class_ = "sidebarBuyingText sidebarWrapper__btn").text
event_price.split("\xa0€")[0]

'14,00'

In [3]:
# 1) Split the data using the "<strong>" tag -> This way we separate the sections
# 2) Split the data using the "</strong>" tag -> This way we separate titles from descriptions
event_descr_items = [elem.split("</strong>") for elem in event_descr.split("<strong>")]

In [4]:
# Just for checking purposes
[len(elem) for elem in event_descr_items]

[1, 2, 2, 2, 2]

In [5]:
# Dict to save the info title-descriptions that we have in a list of lists
new_dict = {}
# This is to assign a numerical value as key to those descriptions without section title
no_title_count = 1

# Iterate over the list of lists, and for every list...
for elem in event_descr_items:
    # If there's more than one element (that means, we have description and title)...
    if len(elem) > 1:
        # Then the first element will be the key and the second one will be the value in our new dict
        new_dict[elem[0]] = elem[1]
    # If we don't have two values (we are missing the title)...
    else:
        # Then, the key will be the numerical value we defined and the value will be the only value of the list, which should be the description
        new_dict[no_title_count] = elem[0]
        no_title_count += 1

new_dict

{1: '<div class="plan-description mb-32" data-cy="plan-content-description">⭐ <b>Meet Vincent van Gogh</b> es una experiencia tridimensional dividida en seis escenarios que permiten adentrarse en la vida de uno de los pintores más famosos de la historia<br/><br/>',
 'Tickets': '<br/>🎫 Entrada - incluye acceso a la exhibición y audioguía<ul><li>General adulto (mayores de 18 años)</li><li>Reducida (diversidad funcional, estudiantes, mayores de 65 años y menores de 18, mostrando siempre documentación)</li><li>Pack de 4 entradas (precio reducido)</li><li>Grupos a partir de 10 personas (precio reducido)</li><li>Menores de 2 años (entrada gratuita)</li></ul><br/>',
 'Qué vas a disfrutar': '<br/>🎨 Descubre una nueva forma de admirar las obras del genio Vincent van Gogh<br/>👨\u200d🎨 Sumérgete en las pinturas más conocidas del artista gracias a una experiencia tridimensional<br/>📽️ Disfruta de una experiencia basada en una de las colecciones más importantes de Van Gogh y en una investigación ex

In [6]:
# This is to remove all the html tags from the text
for key, value in new_dict.items():
    new_dict[key] = re.sub(r"\<.*?\>", "", value)

new_dict

{1: '⭐ Meet Vincent van Gogh es una experiencia tridimensional dividida en seis escenarios que permiten adentrarse en la vida de uno de los pintores más famosos de la historia',
 'Tickets': '🎫 Entrada - incluye acceso a la exhibición y audioguíaGeneral adulto (mayores de 18 años)Reducida (diversidad funcional, estudiantes, mayores de 65 años y menores de 18, mostrando siempre documentación)Pack de 4 entradas (precio reducido)Grupos a partir de 10 personas (precio reducido)Menores de 2 años (entrada gratuita)',
 'Qué vas a disfrutar': '🎨 Descubre una nueva forma de admirar las obras del genio Vincent van Gogh👨\u200d🎨 Sumérgete en las pinturas más conocidas del artista gracias a una experiencia tridimensional📽️ Disfruta de una experiencia basada en una de las colecciones más importantes de Van Gogh y en una investigación exhaustiva sobre su vida🖼️ Una experiencia diseñada por el Van Gogh Museum de Ámsterdam🏴\u200d☠️ Adéntrate en una estimulante actividad con la "Búsqueda del tesoro para 

In [7]:
new_dict["Información General"]

'📅 Fechas: a partir de septiembre de 2021🕒 Horarios:-Lunes a jueves y domingos: de 10:00 a 20:00-Viernes y sábados: de 10:00 a 21:00⏳ Duración: 60 minutos aproximadamente👤 Edad: ¡todas las edades! (menores de 14 años acompañados de un adulto y menores de 24 meses gratis)📍 Lugar: Espacio Ibercaja Delicias - Paseo de las Delicias 61 (Madrid)♿ Accesibilidad: accesible para personas con movilidad reducida❓ Puedes consultar las preguntas frecuentes, sus respuestas y las medidas sanitarias aquí'

In [8]:
# Now I have to do all the processing with the iterators to pull the information from "Información general".

# We'll need an iterator
# iterator = new_dict["Información General"]
info_names = ["Información", "Información General"]
for name in info_names:
    try:
        iterator = new_dict[name]
        break
    except:
        pass

# We'll need an empty list too, save the subsections
general_info_sections = []

# Lastly, we'll make use of an icons' list to check whether our text has any emoji in it or not
# icons_list = emoji.UNICODE_EMOJI["es"].keys()
icons_list = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]

# Iterate over all the icons in the list
for icon in icons_list:
    # If the icon is in the iterator (string)...
    if icon in iterator:
        # Split iterator using the icon and save it as the new iterator
        # Input: string
        # Output: list with 2 items
        iterator = iterator.split(icon)
        # Save the first item in our new list
        general_info_sections.append(iterator[0])

        # Check if there is still any icon left in the second element of the iterator
        if any(icon in iterator[1] for icon in icons_list):
            # If so, save it as the new iterator (a string again)
            iterator = iterator[1]

        # Else, save it in our new list.
        # As we don't have more icons in the second element, that means, we don't need to split the iterator anymore, since we already reach the last piece of info we needed
        else:
            general_info_sections.append(iterator[1])
    else:
        pass

general_info_sections

['',
 ' Fechas: a partir de septiembre de 2021',
 ' Horarios:-Lunes a jueves y domingos: de 10:00 a 20:00-Viernes y sábados: de 10:00 a 21:00',
 ' Duración: 60 minutos aproximadamente',
 ' Edad: ¡todas las edades! (menores de 14 años acompañados de un adulto y menores de 24 meses gratis)',
 ' Lugar: Espacio Ibercaja Delicias - Paseo de las Delicias 61 (Madrid)',
 ' Accesibilidad: accesible para personas con movilidad reducida',
 ' Puedes consultar las preguntas frecuentes, sus respuestas y las medidas sanitarias aquí']

In [9]:
general_info_sections[2].split(":", 1)

[' Horarios',
 '-Lunes a jueves y domingos: de 10:00 a 20:00-Viernes y sábados: de 10:00 a 21:00']

In [10]:
general_info_sections_2 = []

for elem in general_info_sections:
    if len(elem) > 1:
        general_info_sections_2.append(elem.split(":", 1))

general_info_sections_2

[[' Fechas', ' a partir de septiembre de 2021'],
 [' Horarios',
  '-Lunes a jueves y domingos: de 10:00 a 20:00-Viernes y sábados: de 10:00 a 21:00'],
 [' Duración', ' 60 minutos aproximadamente'],
 [' Edad',
  ' ¡todas las edades! (menores de 14 años acompañados de un adulto y menores de 24 meses gratis)'],
 [' Lugar', ' Espacio Ibercaja Delicias - Paseo de las Delicias 61 (Madrid)'],
 [' Accesibilidad', ' accesible para personas con movilidad reducida'],
 [' Puedes consultar las preguntas frecuentes, sus respuestas y las medidas sanitarias aquí']]

In [11]:
sections = {}
extra = 1

for list_ in general_info_sections_2:
    if len(list_) > 1:
        sections[list_[0]] = list_[1]
    else:
        sections[extra] = list_[0]
        extra += 1

sections

{' Fechas': ' a partir de septiembre de 2021',
 ' Horarios': '-Lunes a jueves y domingos: de 10:00 a 20:00-Viernes y sábados: de 10:00 a 21:00',
 ' Duración': ' 60 minutos aproximadamente',
 ' Edad': ' ¡todas las edades! (menores de 14 años acompañados de un adulto y menores de 24 meses gratis)',
 ' Lugar': ' Espacio Ibercaja Delicias - Paseo de las Delicias 61 (Madrid)',
 ' Accesibilidad': ' accesible para personas con movilidad reducida',
 1: ' Puedes consultar las preguntas frecuentes, sus respuestas y las medidas sanitarias aquí'}

In [12]:
general_df = pd.DataFrame(new_dict, index = [0])
sections_df = pd.DataFrame(sections, index = [0])

In [13]:
full_df = pd.merge(general_df, sections_df, how = "outer", left_index = True, right_index = True)
full_df

,1_x,Tickets,Qué vas a disfrutar,Información General,Descripción,Fechas,Horarios,Duración,Edad,Lugar,Accesibilidad,1_y
0,⭐ Meet Vincent van Gogh es una experiencia tri...,🎫 Entrada - incluye acceso a la exhibición y a...,🎨 Descubre una nueva forma de admirar las obra...,📅 Fechas: a partir de septiembre de 2021🕒 Hora...,¿Alguna vez has soñado con adentrarte en una p...,a partir de septiembre de 2021,-Lunes a jueves y domingos: de 10:00 a 20:00-V...,60 minutos aproximadamente,¡todas las edades! (menores de 14 años acompa...,Espacio Ibercaja Delicias - Paseo de las Deli...,accesible para personas con movilidad reducida,"Puedes consultar las preguntas frecuentes, su..."


pd.DataFrame([new_dict, new_dict])